In [36]:
import openai
openai.api_key = "<openAPI key>"

In [38]:
import os 

os.environ["OPENAI_API_KEY"] = "<openAPI key>"

In [16]:
import openai
api_key = "<openAPI key>"
client = openai.OpenAI(api_key=api_key)


def upload_file(training_data_path):
    
    with open(training_data_path,"rb") as file: 
        response = client.files.create(
            file = file,
            purpose="fine-tune"
        )
    return response.id

def create_finetuning(training_data_path, model_name):
    client = OpenAI(api_key= "<openAPI key>")
    training_field_id = upload_file(training_data_path)
    response = client.fine_tuning.jobs.create(training_file=training_field_id, model = model_name)
    print(response.id)


file-HCBXOCFZvvhcNYJDv3RVqM9T


In [4]:
#gan model 
import openai
import csv
from random import randint
from openai import OpenAI
import pandas as pd 
import json
import random 
import tensorflow as tf
import numpy as np

from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix, mean_squared_error
api_key = "<openAPI key>"
client = openai.OpenAI(api_key=api_key)
openai.api_key = "<openAPI key>"
generated =""
real_data = ""
real_output = []
fake_output = []
topics_chosen =[]

def generator(model_name): 

    completion = openai.chat.completions.create(
    model=model_name,
    messages=[
            {
                "role":"system", 
                "content" : "Generated a human-written paragraph"

            },
            {   
                "role": "user",
                "content": "Generate a short essay"
            },
        ],    
        )
    generated = completion.choices[0].message.content
    return generated
    
    
def discriminator(generated, real_data,epoch,model_name,fake_input = False, ):
    if fake_input == True:
        completion = openai.chat.completions.create(

            #model = "ft:gpt-3.5-turbo-1106:personal::8TAxuuym",
            model = model_name,
            messages=[
                {
                    "role": "system", 
                    "content": "Classify the following text as AI-generated or human-written .", 

                },
                {
                    "role":"user", 
                    "content": generated, 
                }
            ]
        )
        print("generated data", generated)
        if"AI-generated"  in completion.choices[0].message.content:
            fake_output.append(1)
        else: 
            fake_output.append(0)
        print("fake output:", fake_output) 

    else: 
        completion = openai.chat.completions.create(

            #model = "ft:gpt-3.5-turbo-1106:personal::8TAxuuym",
            model = model_name,

            messages=[
                {
                    "role": "system", 
                    "content": "Classify the following text as AI-generated or human-written .", 

                },
                {
                    "role":"user", 
                    "content": real_data, 
                }
            ]
        )
        print("\n\nreal_data",real_data)
        if"AI-generated"  in completion.choices[0].message.content:
            real_output.append(1)
        else: 
            real_output.append(0)
        print("real ouput: ", real_output) 


    

    

    
def format_for_gpt(row, generator = False):
   if generator == False :
        prompt =  row['text']
        completion = "This text is " + ("AI-generated." if row['generated'] == 1 else "human-written.")
        return {
            "messages": [
                {"role": "system", "content": "Classify the following text as AI-generated or human-written ."}, 
                {"role": "user", "content": prompt},
                {"role": "assistant", "content": completion}
            ]
        }
   
   else:
        completion = row['text']
        return{
           "messages": [
                {"role": "system", "content": "Generate a human-written like essay"}, 
                {"role": "user", "content": "Write a short essay."},
                {"role": "assistant", "content": completion}
            ]
       }
       

def upload_file(training_data_path):
    
    with open(training_data_path,"rb") as file: 
        response = client.files.create(
            file = file,
            purpose="fine-tune"
        )
    return response.id

def create_finetuning(training_data_id, model_name):
    client = OpenAI(api_key= "<openAPI key>")
    training_field_id = upload_file(training_data_id)
    response = client.fine_tuning.jobs.create(training_file=training_field_id, model = model_name)
    print(response.id)


def discriminator_loss(real_output, fake_output):
    real_loss = mean_squared_error(tf.zeros_like(real_output), real_output)
    fake_loss = mean_squared_error(tf.ones_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss



def generator_loss(fake_output):
    return mean_squared_error(tf.zeros_like(fake_output), fake_output)


#first training loop 

def training_loop(generator_model_name,dis_model,epoch = 0):
    real_data = ""
    generated = ""
    training_loop = 10

    with open(f'discriminator_data_part{epoch +1}.csv', 'w') as file1,  
         open(f'generator_data_part{epoch +1}.csv', 'w') as file2 :

        for i in range(training_loop):
            generated=generator(generator_model_name)
                
            discriminator(generated = ...)
            writer1.writerow([generated, 1])                
                
            data = pd.read_csv('./Filtered_Essay_Data.csv')
            k = random.randint(0,10000)
            real_data = data["text"][k] 
            discriminator(generated, ...)
            writer1.writerow([real_data, 0])
            writer2.writerow([real_data, 0])     
    
        dis_loss = discriminator_loss(real_output, fake_output)
        gen_loss = generator_loss(fake_output)

        print("dis_loss:",dis_loss)
        print("gen_loss:", gen_loss)

        if dis_loss > gen_loss:
            create_finetuning(
                f'./formatted_discriminator_data_part_{epoch+1}.jsonl',
                dis_model)
            print("discriminator is updated")

        else:
            create_finetuning(
                f'./formatted_generated_data_part_{epoch+1}.jsonl',
                generator_model_name)
            print("generator is updated")
        
            








In [72]:
#first training loop :
training_loop("gpt-3.5-turbo","gpt-3.5-turbo",0)

In [70]:

#2nd training loop 
training_loop("gpt-3.5-turbo","ft:gpt-3.5-turbo-0613:personal::8UwDvuTO",1)

In [12]:

#3rd training loop 
training_loop("ft:gpt-3.5-turbo-0613:personal::8Uy7r639","ft:gpt-3.5-turbo-0613:personal::8UwDvuTO",2)

fake output: [1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0]
Venus is the planet of our solar system. Occasionally Venus is the closest planet to Earth in term of size as well as distance. Venus, also known as "Evening Star," is a second planet from our sun that makes it more brighter and warmer. Venus is covered with a large blanket of carbon dioxide. The temperature on the Venus is approximately 800 degrees Fahrenhite, and also has the atmospheric pressure 90 time greater than on Earth, this circumstances make the Human life quite impposible on Venus. Venusian geology has impediments like volcano eruption and earthquickes.

Do you ever feel how interesting it would be to visit places that are out of this world? Or, do you ever thought, what if we get the chance to visit different planets like Venus or Mars? Well, we are no longer away from the time when we will visit different planets for our vacation trips. Although our neighboring planets are inhospitable of human life, however ; Astronomers are

In [14]:
#4th training loop 
training_loop("ft:gpt-3.5-turbo-0613:personal::8Uy7r639","ft:gpt-3.5-turbo-0613:personal::8UyJm4hV",3)

generated data I am just going to run to the store to get some stuff to make dinner tonight before the snow storm comes.I but the bread i needed justine serrano making spaghetti and garlic bread for me and my son. All of a sudden i looked outside and these big, white, unwanted, dangerous snow flakes started to come out. I was not happy at all. It was not time for snow, i had things to do and places to be and this snow was not going to make it happen. I was in a big rush and just wanted to hurry up and get out of the store and get home. While i was waiting in line a costumer had the nerve to speak up and explain to everyone that i was in a hurry to go home and cook dinner for me and my son. I ended up pissing me off more because it was true and then i relized that it was probley because it was all my fault! But anyways got the stuff and came home. I made the spaghetti and the garlic bread and me and my son both ate. i did end up going outside and attempted to get my car out but that did

In [5]:
#5th training loop 
training_loop("ft:gpt-3.5-turbo-0613:personal::8Uy7r639",'ft:gpt-3.5-turbo-0613:personal::8Uz5IfPq',4)

generated data As a resident of the Bahamas, one must consider the fact that the Bahamas is susceptible to hurricanes, which is a natural disaster cannot be stopped by people. Although hurricanes devastate residents each year, there are advantages to living in the Bahamas, such as the breathtaking scenery, the friendly locals, the beautiful water, the government's response time to natural disasters, and how the government helps the residents after a natural disaster.

The Bahamas is known for being the most beautiful place on the earth, with beaches and breathtaking scenery. A day on the beach in the Bahamas is not like a day on the beach in America; it is more private and peaceful. Also, the color of the Bahamian water is the first feature that catches an American's attention because it is more blue. There is also a taste to the water, but it is not salty, it has more of a flavor.

Another advantage of living in the Bahamas is that the Bahamian citizens are very welcoming and friendly

In [12]:
#6th training loop 

training_loop("ft:gpt-3.5-turbo-0613:personal::8Uy7r639","ft:gpt-3.5-turbo-0613:personal::8V9qL2GK",5)

generated data This is an essenstial may people disrespect employed students. Students need to be motivated to go to school everyday. This leads to money difficulties or problems that can cause them to work late,miss meals or even live without heat,air conditioners,or electricity.Sometimes parents bring in too little money, other times they get laid off. Most of the time employed students are missing meals because did you them of money but they still have ot help out with bills. They can be feeding their siblings and even paying bills that shout be otay their parents resposibilty.High school students do not wanna feel bad though so they do not tell their friends. When you are a high school student you needs friends to make it by great and this cause them to keep it a secret.  The first cause of employed students missing meals is that their parents take money out of their check affecting the child. Writing checks for rent, watr, light, and gas would make the child miss a meal. The would

In [17]:
#7th training loop 

training_loop("ft:gpt-3.5-turbo-0613:personal::8VAQL3Rz","ft:gpt-3.5-turbo-0613:personal::8V9qL2GK",6)

generated data In the article "Should We Use $700 Computers to Help Us Learn Math?", a math teacher Mark Davidson is trying to explain to the school bored at his middle school that computers would really help the students out by having one themselves. Davidson gose and checks out the computer before he ask's the school bored. He uses the software by Stanford Mathematician Conrad Wolfram he wrote to teach his young daughter math. In the artice Davidson quotes "I'm doing math constantly, but all I'm really doing is supplying a tool for math doing-and-learning that's personal." The reason why the teacher is all for the computers is because a child can go out and get a personal math tool like a calculator to use in class to do math, but you wouldn't take out your calculator to do a quick math problem on the go. The student can take out his computer to do the math problem quickly. The artice suggests that the students should get a computer.

I think that the students should be aloud to reci

In [3]:
# 8th training loop 

training_loop("ft:gpt-3.5-turbo-0613:personal::8VFDIw3o","ft:gpt-3.5-turbo-0613:personal::8V9qL2GK",7)

generated data People should not have cars, is becuse it analysis a lot of problems wen use it, people have a lot of reason by usnig it, and people dresser can be diffrid just by tips diey have with them using a car.

Firstly, one of the resons why we should not use car is becuse it analysis a lot of problems when wasing it, like out us or space snd the most important things that not only we have when we uses the car but also when we use the bus, the car has anon advantage. For example, if on a car crash one of the passenger get a problme it is very hard to find the solution to cure him, and it is more dangrous to use the car because of air pollution, noise pollution and water pollution.

Secondly, one of the reasons with that people are familiar is a lot by usin the car that we change our transportation, is very harms would days because they not longer saw ther friends becuse they did not how they was go to ther house but the population whit that is going down now using something that

In [6]:
#9 th training loop 

training_loop("ft:gpt-3.5-turbo-0613:personal::8VFVQzDx","ft:gpt-3.5-turbo-0613:personal::8V9qL2GK",8)

generated data The Venusians... Face of Mars... Martians??? Well according to Dr . Kia it's completely plausible  that there is life on Venus mean heck people live in "one of the most similar places no earth." These people (7,000,000+) seem to be doing fine with how they live. They don't sufficate to the presence of CO2 all day like we sufficate the type of car we see on a daily basis. What makes the newspapers, T.V's, news stations, anything claiming there are Venusians, truthul? Same with the martian operation, what solid proof is there that they did find bacteria on Mars. People just believe it's official beacuse of an article in a newspaper which honestly who knows where all the the correct facts are. Let's not be so quick to believe the hype of Venusians, Aliens, Martians w/e other type of life form like people on the Venus campaign trail. Life outside of earth is very plausible.

What even are Venusians? Aliens, little Venus people? Is "Venusians" even a word? Who knows? Is it ju

In [8]:
#10th training loop 

training_loop("ft:gpt-3.5-turbo-0613:personal::8VFpGwTc","ft:gpt-3.5-turbo-0613:personal::8V9qL2GK",9)

generated data The Mystery of the Facial Cygnus appears to be the most heart stopping discovery of all time. In reality, it is merely a coincidence of sorts, nothing more than an optical illusion. On pages 26 and 27 of the November 1998 issue of VAN, you will find photos of the Ausurian stones taken by the Austrian author known as Robert Bauval. These photos were paired with another photo, and from this, people have discovered the face is not a supernatural structure created by aliens past ancestors but instead an illusion created by light and shadows.

The Pharos of Heliopolis or Haracekh is what people belive to be the true cause of the fashinating find. Additonal research and evedence shows that the Arabs who settled this part of the world admire a famous Arabic phrase which speaks of the "Moutain of Pharaoh that the sea once covered bridges on". Amusingly enough, there are no ancient Egyptian structures of any kind in the Usurian Syatem. Most incridbly is that the supposive "astron

In [10]:
#evalution of final model (discriminator)

import pandas as pd

# Load the CSV file
file_path = './Training_Essay_Data.csv'  # Replace with your file path
data = pd.read_csv(file_path)

# Split the data into generated and non-generated groups
generated_data = data[data['generated'] == 1]
non_generated_data = data[data['generated'] == 0]

# Randomly sample 25 entries from each group
sampled_generated = generated_data.sample(25)
sampled_non_generated = non_generated_data.sample(25)

# Combine the two samples
combined_sample = pd.concat([sampled_generated, sampled_non_generated])

# Save the combined sample to a new CSV file
output_file_path = './Sampled_Essay_Data.csv'  # Replace with your desired output file path
combined_sample.to_csv(output_file_path, index=False)



In [14]:
import numpy as np
from sklearn.metrics import mean_squared_error, confusion_matrix, matthews_corrcoef
# Create an array with 25 ones and 25 zeros
ones = np.ones(25, dtype=int)
zeros = np.zeros(25, dtype=int)
true_label = np.concatenate((ones, zeros))

print(true_label)
predicted_label = []
data = pd.read_csv("./Sampled_Essay_Data.csv")

for i in data["text"]:
    completion = openai.chat.completions.create(

            model = "ft:gpt-3.5-turbo-1106:personal::8TAxuuym",
        
            messages=[
                {
                    "role": "system", 
                    "content": "Classify the following text as AI-generated or human-written .", 

                },
                {
                    "role":"user", 
                    "content": i, 
                }
            ]
        )
    if"AI-generated"  in completion.choices[0].message.content:
        predicted_label.append(1)
    else: 
        predicted_label.append(0)
print("output:", predicted_label) 


mse_score = mean_squared_error(true_label,predicted_label)
print("mse score:", mse_score)

confusion_m = confusion_matrix(true_label, predicted_label)
print("confusion_matrix", confusion_m)

matthews_corrcoef_score = matthews_corrcoef(true_label, predicted_label)
print("matthew corrcoef:", matthews_corrcoef_score)



[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
output: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
mse score: 0.02
confusion_matrix [[25  0]
 [ 1 24]]
matthew corrcoef: 0.9607689228305227


In [1]:
from rouge import Rouge 

def calculate_rouge_score(hypothesis, reference):
    rouge = Rouge()
    scores = rouge.get_scores(hypothesis, reference)
    return scores

hypothesis = "When you think about all the necessary parts that our society is made of in order for it to function, an unexpected image might come to your mind: a classroom. Education is not only a building process of knowledge, but it is the cornerstone of development and limitless progress. In fact, this road to innovation and the grounding for all future successes begins at the same, monotonous place where you learn your ABCs. The twenty-six letters of our alphabet are the same ones that allow you to learn and think critically. Thinking is what allows us to go beyond the status quos and what permits us to find new methods to almost every diverse obstacle. But, with the increasing advancements in the modern world, is education still as important as it was in the past? The answer is quite simple, Absolutely! Imagine for a moment what your life would be like if you couldnt read, or write, or even process new information. In just that instance alone, the world would seem so much smaller than what it is today. The seemingly universal language that is formed by most individuals, spoken words, only covers a small percentage of the global population. From your first steps, to your first day of high school, and even your first job, you use a form of communication to express your thoughts and feelings on a certain subject. Thinking critically often helps you form an opinion or stance on an issue, and then, communication steps in. You are then able to express to others your thoughts and ideas, and expand upon them. Without thinking and talking about something, an idea is just that, an idea, a figment of our imaginations that was never given the light of being a reality. Now imagine if the idea was a solution for climate change or a ground-breaking innovation that would advance society at a level never dreamed before. Without critical thinking and the ability to communicate those thoughts education, these ideas and many more would be forever lost to ignorance. The false perception that education holds little value in comparison to more seemly important issues is a grave disease that manifests itself in impoverished fields and unreached places throughout the world. This disease is an epidemic that is stopped by one thing, the very thing it often hinders, education. Education is the souls sustenance. Education empowers you to empower others. But of course, society is always searching for the bell of innovation, that awakens the sleeping giant that is success. If that be true, how do you always achieve innovations and improvements? Is it through the construction of high tech laboratories or the secrets that are held within the walls of Silicon Valley? Yes, those do aid in the advancement of society, but they are not the root of the development that we all seek. Inside each and every one of you, there are an abundance of actions and decisions that are just asking to change and better the world. Education is just the chisel to help whittle away the excess marble to uncover the hidden statue that is positive change. And like any artist will tell you, to create appealing art, the chisel must be sharp, and one must be educated in their craft. Even those who aspire to become entrepreneurs must first embrace the empowering qualities of education, before they can even begin to think about becoming a Steve Jobs or Mark Zuckerberg. Just like a dwarf planet is still a planet, limitless progress must first conquer the roadblocks that are ignorance, and replace them with the very steps that will pave the road to advance. Education is not only the weight upon our youthful shoulders, it is the bridge whose construction will interconnect the different sectors of the world, such as poverty reduction and gender equality, into a sustainable, innovative society. A society that is more than just educated, but is an education."
reference = "Title: The Power of Education\n\nEducation is a fundamental pillar of society, providing individuals with the tools and knowledge necessary to lead fulfilling lives. Its impact extends far beyond classroom walls, shaping not only individual lives but also the development and progress of nations. The power of education lies in its ability to empower individuals, eradicate ignorance, foster critical thinking, and drive positive change.\n\nFirstly, education empowers individuals by equipping them with essential skills and knowledge. It provides the necessary tools to navigate the complexities of the modern world and enhances one's ability to make informed decisions. Education empowers individuals to pursue their passions and interests, opening up a world of possibilities and opportunities. It enables individuals to secure better employment prospects, achieve economic stability, and contribute actively to their communities.\n\nFurthermore, education is the key to eradicating ignorance and challenging prejudiced beliefs. It exposes individuals to a diverse range of perspectives, cultures, and ideas, enabling them to develop a broader understanding of the world. This exposure promotes tolerance, empathy, and acceptance, fostering a more inclusive and harmonious society. By dispelling ignorance, education stands as a powerful force against discrimination, bigotry, and social inequality.\n\nMoreover, education fosters critical thinking, enabling individuals to question and analyze the world around them. It encourages individuals to develop an inquisitive mindset, seek knowledge beyond curriculum boundaries, and form their own opinions. Critical thinking equips individuals with the capacity to challenge existing norms, address complex issues, and find innovative solutions. It is through education that societies cultivate the next generation of thinkers, leaders, and problem-solvers.\n\nLastly, education drives positive change on both an individual and societal level. As educated individuals contribute to their communities, possessing the knowledge and skills to tackle social and economic challenges, they become agents of progress. Education is a catalyst for socio-economic development, fostering innovation, entrepreneurship, and sustainable growth. It plays a pivotal role in reducing poverty, improving healthcare, and promoting gender equality.\n\nIn conclusion, education has the power to transform lives and societies. It provides individuals with the means to empower themselves, eradicate ignorance, foster critical thinking, and drive positive change. As we continue to prioritize and invest in education, we can create a more equitable, enlightened, and prosperous world for future generations."

scores = calculate_rouge_score(hypothesis, reference)
print(scores)

[{'rouge-1': {'r': 0.2981651376146789, 'p': 0.1906158357771261, 'f': 0.23255813477696252}, 'rouge-2': {'r': 0.05555555555555555, 'p': 0.031096563011456628, 'f': 0.03987407724517207}, 'rouge-l': {'r': 0.2706422018348624, 'p': 0.17302052785923755, 'f': 0.21109122958912713}}]


The output is a list of dictionaries containing the ROUGE scores for comparing two pieces of text.

- `rouge-1`, `rouge-2`, and `rouge-l` are different types of ROUGE scores:
  - `rouge-1` refers to the overlap of unigrams (single words) between the generated text and the reference text.
  - `rouge-2` refers to the overlap of bigrams (two consecutive words) between the generated text and the reference text.
  - `rouge-l` refers to the longest common subsequence (LCS) between the generated text and the reference text. The LCS is the longest sequence of words that are the same between the generated text and the reference text, in the same order, but not necessarily consecutive.

- `r`, `p`, and `f` are different measures of the overlap:
  - `r` stands for recall, which is the proportion of the reference text that is also in the generated text.
  - `p` stands for precision, which is the proportion of the generated text that is also in the reference text.
  - `f` stands for F1 score, which is the harmonic mean of precision and recall. It provides a single score that balances both precision and recall.

In the output, for example, the `rouge-1` recall is 0.298, meaning that about 29.8% of the words in the reference text are also in the generated text. The `rouge-1` precision is 0.190, meaning that about 19.0% of the words in the generated text are also in the reference text. The `rouge-1` F1 score is 0.232, providing a single score that balances both the precision and recall.